# Qubit spectroscopy: $T_1$ measurement example

This notebook demonstrates how to assess the decoherence time, $T_1$, of one or more qubits on a real quantum device using pyQuil. A $T_1$ experiment consists of an X pulse, bringing the qubit to the excited state, followed by a delay of variable duration. We expect the state to decay exponentially with increasing time, and we characterize this decay by the time decay constant, which we call $T_1$ and refer to as the qubit's "decoherence time." A sample QUIL program at one data point (specified by the duration of the DELAY pragma) for qubit 0 with a 10us wait would look like 

    DECLARE ro BIT[1]
    RX(pi) 0
    PRAGMA DELAY 0 "1e-05"
    MEASURE 0 ro[0]

**NB: Since decoherence and dephasing noise are only simulated on gates, and we make use of DELAY pragmas to simulate relaxation time, we cannot simulate decoherence on the QPU with this experiment as written. This notebook should only be run on a real quantum device.**

## setup - imports and relevant units

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from pyquil.api import get_qc, QuantumComputer

from forest_benchmarking.qubit_spectroscopy import (
    exponential_decay_curve,
    fit_to_exponential_decay_curve,
    run_t1)

We treat SI base units, such as the second, as dimensionless, unit quantities, so we define relative units, such as the microsecond, using scientific notation. 

In [ ]:
MICROSECOND = 1e-6

## measurement, analysis, and plotting

This method takes a `pandas.DataFrame` object with all the $T_1$ data, including the times and qubits measured and the resulting decay at each data point, and plots the decay of each qubit with respect to increasing delay duration. It also fits to an exponential decay curve, evaluates the fitted time decay constant, $T_1$, for each qubit, and plots the fitted curve on top of the experimental data so that a user can assess the fit if so desired. 

In [ ]:
def plot_fitted_exponential_decay_curve_over_data(df: pd.DataFrame,
                                                  filename: str = None) -> None:
    """
    Plot T1 experimental data and fitted exponential decay curve.

    :param df: Experimental results to plot and fit exponential decay curve to.
    :param qc_type: String indicating whether QVM or QPU was used to collect data.
    :return: None
    """
    for q in df['qubit'].unique():
        df2 = df[df['qubit'] == q].sort_values('time')
        x_data = df2['time']
        y_data = df2['avg']

        plt.plot(x_data, y_data, 'o-', label=f"QC{q} T1 data")

        try:
            fit_params, fit_params_errs = fit_to_exponential_decay_curve(x_data, y_data)
        except RuntimeError:
            print(f"Could not fit to experimental data for QC{q}")
        else:
            plt.plot(x_data, exponential_decay_curve(x_data, *fit_params),
                     label=f"QC{q} fit: T1={fit_params[1] / MICROSECOND:.2f}us")

    plt.xlabel("Time [s]")
    plt.ylabel("Excited state visibility")
    plt.title("T1 decay")

    plt.legend(loc='best')
    plt.tight_layout()
    if filename is not None:
        plt.savefig(filename)
    plt.show()

First, we collect our $T_1$ raw data using `run_t1`. We can specify which qubits we want to measure using `qubits` and the maximum delay we'll use for each using `stop_time`. 

In [ ]:
# qc = get_qc('Aspen-1-3Q-A')
qc = get_qc('2q-noisy-qvm') # will run on a QVM, but not meaningfully 
qubits = qc.qubits()

df = run_t1(qc=qc, 
            qubits=qubits, 
            stop_time=60 * MICROSECOND,
            filename=f't1.{qc.name}.json')

Now we can use the above plotting method to measure $T_1$ on our lattice, `Aspen-1-3Q-A`. 

In [ ]:
plot_fitted_exponential_decay_curve_over_data(df, filename=f't1.{qc.name}.png')